In [2]:
import requests
import xml.etree.ElementTree as ET


from io import StringIO

from LoadingData import LoadFromAPI
import os
import pandas as pd
import numpy as np

## Air Quality API: 空气质量接口：

help网页： [https://api.erg.ic.ac.uk/AirQuality/help](https://api.erg.ic.ac.uk/AirQuality/help)

### Air Quality - MonitoringLocalAuthority

In [48]:
LoadFromAPI.load_data_from_api(url='https://api.erg.ic.ac.uk/AirQuality/Information/MonitoringLocalAuthority/GroupName=London', data_format="xml", filename="Data/AirQuality/MonitoringLocalAuthority.xml")

Data saved to Data/AirQuality/MonitoringLocalAuthority.xml.


In [49]:
# 解析xml文件并且提取有用信息，存储到dataframe中并且保存到csv文件中


tree = ET.parse('Data/AirQuality/MonitoringLocalAuthority.xml')
root = tree.getroot()

Autho_data = []
for record in root.findall('LocalAuthority'):
    name = record.get('LocalAuthorityName')
    code = record.get('LocalAuthorityCode')
    latitude = record.get('LaCentreLatitude')
    longitude = record.get('LaCentreLongitude')
    Autho_data.append([name, code])

Autho_df = pd.DataFrame(Autho_data, columns=['LocalAuthorityName', 'LocalAuthorityCode'])

### Air Quality - MonitoringSites

In [50]:
#url = http://api.erg.ic.ac.uk/AirQuality/Information/MonitoringSites/GroupName={GROUPNAME}

In [51]:
LoadFromAPI.load_data_from_api(url='http://api.erg.ic.ac.uk/AirQuality/Information/MonitoringSites/GroupName=London', data_format="xml", filename="Data/AirQuality/MonitoringSites.xml")

Data saved to Data/AirQuality/MonitoringSites.xml.


In [52]:
# XML文件路径

# 解析XML文件
tree = ET.parse('Data/AirQuality/MonitoringSites.xml')
root = tree.getroot()

# 提取数据
data = []
for site in root.findall('Site'):
    # 获取属性
    local_authority_code = site.get('LocalAuthorityCode')
    local_authority_name = site.get('LocalAuthorityName')
    site_code = site.get('SiteCode')
    site_name = site.get('SiteName')
    site_type = site.get('SiteType')
    date_opened = site.get('DateOpened')
    date_closed = site.get('DateClosed')
    latitude = site.get('Latitude')
    longitude = site.get('Longitude')
    
    # 将提取的数据添加到列表中
    data.append({
        'LocalAuthorityCode': local_authority_code,
        'LocalAuthorityName': local_authority_name,
        'SiteCode': site_code,
        'SiteName': site_name,
        'SiteType': site_type,
        'DateOpened': date_opened,
        'DateClosed': date_closed,
        'Latitude': latitude,
        'Longitude': longitude
    })

# 创建DataFrame
AQ_Sites_df = pd.DataFrame(data)

In [53]:
# save to csv
AQ_Sites_df.to_csv('Data/AirQuality/AQ_Sites.csv', index=False)

### Air Quality - Species

In [54]:
LoadFromAPI.load_data_from_api(url='https://api.erg.ic.ac.uk/AirQuality/Information/MonitoringSiteSpecies/GroupName=London', data_format="xml", filename="Data/AirQuality/MonitoringSiteSpecies.xml")

Data saved to Data/AirQuality/MonitoringSiteSpecies.xml.


In [ ]:
# Parsing the XML content
tree = ET.parse('Data/AirQuality/MonitoringSiteSpecies.xml')
root = tree.getroot()

# Collecting data into a list of dictionaries
data = []

# Iterate over each site in the XML
for site in root.findall('Site'):
    site_data = site.attrib  # Get the site attributes
    for species in site.findall('Species'):
        # Combine site data with species data for each row
        species_data = species.attrib
        row_data = {**site_data, **species_data}
        data.append(row_data)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('Data/AirQuality/MonitoringSiteSpecies.csv', index=False)

In [ ]:
# Try one
LoadFromAPI.load_data_from_api(url='https://api.erg.ic.ac.uk/AirQuality/Annual/MonitoringObjective/SiteCode=BG1/StartDate=01 Jan 2020', 
                               data_format="xml", 
                               filename="Data/AirQuality/MonitoringReport_BG1_2022.xml")




Data saved to Data/AirQuality/MonitoringReport_BG1_2022.xml.


In [3]:
LoadFromAPI.load_data_from_api(url='https://api.tfl.gov.uk/AccidentStats/2022', 
                               data_format="json", 
                               filename="Data/AirQuality/AccidentStats_2022.json")


Failed to fetch data. Status code: 400


get the specific species information of specific site during some period fromdate to enddate from the API

This returns raw data based on 'SiteCode', 'StartDate', 'EndDate'. Default time period is 'hourly'. Data returned in CSV format

"https://api.erg.ic.ac.uk/AirQuality/Data/Site/Wide/SiteCode={SITECODE}/StartDate={STARTDATE}/EndDate={ENDDATE}/csv"

In [ ]:
LoadFromAPI.load_data_from_api(url='https://api.erg.ic.ac.uk/AirQuality/Data/SiteSpecies/SiteCode=BQ7/SpeciesCode=PM25/StartDate=01 Mar 2022/EndDate=01 Apr 2023', 
                               data_format="xml", 
                               filename="Data/AirQuality/BQ7_20180101.xml")

Data saved to Data/AirQuality/BQ7_20180101.xml.


In [3]:
LoadFromAPI.load_data_from_api(url='https://api.erg.ic.ac.uk/AirQuality/Data/SiteSpecies/SiteCode=BQ7/SpeciesCode=PM25/StartDate=19 Apr 2022/EndDate=01 Apr 2023/csv',
                               data_format="csv", 
                               filename="Data/AirQuality/BQ7_20180101.csv") 
                               

Data saved to Data/AirQuality/BQ7_20180101.csv.


**pivot the long format data to wide format data**

In [ ]:
# read data from csv file
df = pd.read_csv('Data/AirQuality/BQ7_20180101.csv')
print(df.columns)

MeasurementDateGMT = df.columns[0]
print(MeasurementDateGMT)
field = df.columns[1]
print(field)

Index(['MeasurementDateGMT', 'Bexley - Belvedere West: PM2.5 Particulate (ug/m3)'], dtype='object')
MeasurementDateGMT
Bexley - Belvedere West: PM2.5 Particulate (ug/m3)


In [ ]:

# Convert the 'MeasurementDateGMT' to datetime to extract date and hour
df['MeasurementDateGMT'] = pd.to_datetime(df['MeasurementDateGMT'])

# Extract hour as a string with format 'HH:MM' to use as column names
df['hour'] = df['MeasurementDateGMT'].dt.strftime('%H:%M')

# Create a date column from the 'MeasurementDateGMT' datetime
df['date'] = df['MeasurementDateGMT'].dt.date

# Pivot the DataFrame to get the wide format with date as index
df_wide = df.pivot(index='date', columns='hour', values='Bexley - Belvedere West: PM2.5 Particulate (ug/m3)')

# 查看随机几行数据
print(df_wide.sample(15))


hour        00:00  01:00  02:00  03:00  04:00  05:00  06:00  07:00  08:00  \
date                                                                        
2022-05-04   13.3   13.9   15.4   20.2   21.3   20.5   22.7   28.3   31.9   
2022-03-14    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2022-10-29    6.8    5.3    5.3    6.0    6.3    6.0    6.3    6.7    7.9   
2022-03-23    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2022-04-22    8.3    9.7   11.6   12.7   13.4   14.9   16.8   15.4   14.8   
2022-11-18    3.0    2.8    2.9    2.7    2.5    2.4    2.5    2.7    2.9   
2023-02-09   23.8   23.8   23.0   22.4   21.5   20.4   18.1   18.4   18.9   
2022-12-17   36.3   34.0   30.4   27.4   26.6   25.2   17.1    4.4    4.6   
2023-01-10    8.9    8.3    7.8    7.6    7.1    7.4    7.9    7.8    5.4   
2023-01-13    6.7    5.3    6.1    6.2    6.8    6.4    6.1    6.5    6.5   
2022-11-23    6.4    5.9    5.5    4.7    3.6    4.0    4.7    4.4    2.0   

In [ ]:
LoadFromAPI.load_data_from_api(url='https://api.erg.ic.ac.uk/AirQuality/Data/SiteSpecies/SiteCode=BT8/SpeciesCode=NO2/StartDate=01 Mar 2022/EndDate=01 Apr 2023/csv', 
                               data_format="csv", 
                               filename="Data/AirQuality/BQ7_20180101.csv")

Data saved to Data/AirQuality/BQ7_20180101.csv.


In [ ]:
# This returns raw data based on 'SiteCode', 'StartDate', 'EndDate'. Data returned in JSON format from traffic counters.
LoadFromAPI.load_data_from_api(url='https://api.erg.ic.ac.uk/AirQuality/Data/Traffic/Site/SiteCode=BT8/StartDate=01 Mar 2022/EndDate=01 Apr 2023/Json', 
                               data_format="json", 
                               filename="Data/AirQuality/BT8_2022_2023.json")

### Potential thing for air quality data

https://www.data.gov.uk/dataset/ef87da6c-0b01-4717-aab4-a076e8b8ff7e/london-atmospheric-emissions-inventory-laei-2013

# London Roads Data

[London Roads Data](https://roads.data.tfl.gov.uk/)

In [2]:
import json

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd


In [ ]:
geojson_file_path = "Data/GLA_TLRN_HAB.geojson"

# 加载GeoJSON数据
gdf = gpd.read_file(geojson_file_path)

# 打印数据帧的前几行，查看数据结构
print(gdf.head(32))

    fid  HA_ID               Borough  Version  Changed_on  \
0     5     19               BROMLEY        1  2007-08-15   
1   106     18                BEXLEY        3  2014-08-01   
2   117     16    BARKING & DAGENHAM        3  2015-05-05   
3   122     21                SUTTON        9  2015-05-05   
4   139      2                CAMDEN        6  2016-05-25   
5   147     12  KENSINGTON & CHELSEA        4  2016-05-25   
6   151     22                MERTON        7  2016-05-25   
7   152     24  RICHMOND UPON THAMES        4  2016-05-25   
8   168      5         TOWER HAMLETS        7  2016-10-24   
9   170     14             REDBRIDGE        4  2016-10-24   
10  176     26            HILLINGDON        2  2017-04-14   
11  177     28                 BRENT        3  2017-04-18   
12  183     15              HAVERING        7  2017-07-03   
13  189      6             GREENWICH        5  2018-03-16   
14  193      8             SOUTHWARK       15  2018-03-21   
15  196     20          

## Interactive GUI for London Roads Data